# The credit fraud dataset - Synthesizing the minority class
In this notebook it's presented a practical exercise of how to use the avilable library GANs to synthesize tabular data.
For the purpose of this exercise it has been used the Credit Fraud dataset from Kaggle, that you can find here:https: //www.kaggle.com/mlg-ulb/creditcardfraud

In [14]:
import importlib

import pandas as pd
import numpy as np
import sklearn.cluster as cluster

from models.gan import model
importlib.reload(model)

from models.gan.model import GAN
model = GAN

In [8]:
#Read the data
data = pd.read_csv('data/data_processed.csv', index_col=[0])
data_cols = list(data.columns[ data.columns != 'Class' ])
label_cols = ['Class']

print('Dataset columns: {}'.format(data_cols))
sorted_cols = ['V14', 'V4', 'V10', 'V17', 'Time', 'V12', 'V26', 'Amount', 'V21', 'V8', 'V11', 'V7', 'V28', 'V19', 'V3', 'V22', 'V6', 'V20', 'V27', 'V16', 'V13', 'V25', 'V24', 'V18', 'V2', 'V1', 'V5', 'V15', 'V9', 'V23', 'Class']
data = data[ sorted_cols ].copy()

Dataset columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


In [9]:
#For the purpose of this example we will only synthesize the minority class
train_data = data.loc[ data['Class']==1 ].copy()

print("Dataset info: Number of records - {} Number of varibles - {}".format(train_data.shape[0], train_data.shape[1]))

algorithm = cluster.KMeans
args, kwds = (), {'n_clusters':2, 'random_state':0}
labels = algorithm(*args, **kwds).fit_predict(train_data[ data_cols ])

print( pd.DataFrame( [ [np.sum(labels==i)] for i in np.unique(labels) ], columns=['count'], index=np.unique(labels) ) )

fraud_w_classes = train_data.copy()
fraud_w_classes['Class'] = labels

Dataset info: Number of records - 492 Number of varibles - 31
   count
0    357
1    135


In [16]:
#Define the GAN and training parameters
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 1000
learning_rate = 5e-4
models_dir = './cache'

train_sample = fraud_w_classes.copy().reset_index(drop=True)
label_cols = [ i for i in train_sample.columns if 'Class' in i ]
data_cols = [ i for i in train_sample.columns if i not in label_cols ]
train_sample[ data_cols ] = train_sample[ data_cols ] / 10 # scale to random noise size, one less thing to learn
train_no_label = train_sample[ data_cols ]

gan_args = [batch_size, learning_rate, noise_dim, train_sample.shape[1], dim]
train_args = ['./cache', epochs, log_step, '']

In [17]:
#Training the GAN model chosen: Vanilla GAN, CGAN, DCGAN, etc.
synthesizer = model(gan_args)
synthesizer.train(train_sample, train_args)

0 [D loss: 0.716243, acc.: 43.75%] [G loss: 0.647072]
generated_data
1 [D loss: 0.673969, acc.: 50.00%] [G loss: 0.661306]
2 [D loss: 0.649647, acc.: 50.00%] [G loss: 0.673505]
3 [D loss: 0.642885, acc.: 50.00%] [G loss: 0.695522]
4 [D loss: 0.637001, acc.: 50.00%] [G loss: 0.735539]
5 [D loss: 0.651138, acc.: 50.39%] [G loss: 0.751465]
6 [D loss: 0.705646, acc.: 50.00%] [G loss: 0.739906]
7 [D loss: 0.809797, acc.: 49.22%] [G loss: 0.675067]
8 [D loss: 0.856178, acc.: 44.53%] [G loss: 0.705571]
9 [D loss: 0.794763, acc.: 16.41%] [G loss: 0.889026]
10 [D loss: 0.690325, acc.: 51.17%] [G loss: 1.180811]
11 [D loss: 0.643485, acc.: 52.34%] [G loss: 1.330645]
12 [D loss: 0.632706, acc.: 55.08%] [G loss: 1.262061]
13 [D loss: 0.640438, acc.: 57.03%] [G loss: 1.170083]
14 [D loss: 0.634663, acc.: 64.84%] [G loss: 1.099782]
15 [D loss: 0.631170, acc.: 71.88%] [G loss: 1.023942]
16 [D loss: 0.640533, acc.: 82.42%] [G loss: 0.922991]
17 [D loss: 0.640538, acc.: 76.17%] [G loss: 0.853670]
18 [D

In [ ]:
#Check the results obtained by the model here


